In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import clear_output
import scipy.io
import os

### Get the relative position for just one timestep:

In [2]:
#Take an entire X_pos and Y_pos array, get the relative position for each X and Y.
def get_relative_positions(X_pos, Y_pos, index):
    rel_X = X_pos - np.reshape(X_pos.T[index], [X_pos.shape[0], -1])
    rel_Y = Y_pos - np.reshape(Y_pos.T[index], [Y_pos.shape[0], -1])
    #Experimental: Theoretically, the exact position of the particle should not be an indicator of its type, or even
    #play any helpful effect along with the relative positions, so we delete it from our inputs.
    rel_X = np.delete(rel_X, index, 1)
    rel_Y = np.delete(rel_Y, index, 1)  
    return rel_X, rel_Y

### Do relative positions for all timesteps in a full simulation:

In [5]:
def transform_to_relative(X_pos, Y_pos, ivel):
    all_rel_X = np.array([])
    all_rel_Y = np.array([])
    labels = np.array([])
    for index in range(X_pos.shape[1]):
        rel_X, rel_Y = get_relative_positions(X_pos, Y_pos, index)
        labels = np.append(labels, np.repeat(ivel[index], X_pos.shape[0]))
        all_rel_X = np.reshape(np.append(all_rel_X, rel_X), [-1, rel_X.shape[1]])
        all_rel_Y = np.reshape(np.append(all_rel_Y, rel_Y), [-1, rel_Y.shape[1]])
    all_rel_pos = np.concatenate([all_rel_X, all_rel_Y], axis = 1)
    labels = np.reshape(labels, [all_rel_pos.shape[0], 1])
    return all_rel_pos, labels

### Relative positions for many simulations shuffled together:

In [6]:
#Get the list of all files in the specified folder, and shuffle them to generalize:
folder = np.array(os.listdir('CoPhe_Lab/1st_May'))
folder = folder[np.random.permutation(len(folder))]
folder_name = "Cophe_Lab\\1st_May\\"

#Specify number of files to process:
file_num = 10
print("Files to process = ", file_num)
_ = input("Press Enter to continue")

file = scipy.io.loadmat(folder_name + folder[0])

#Initialize the array for the XA and YA positions
all_Xpos = np.array(file['XA']).T
all_Ypos = np.array(file['YA']).T

#Get the Velocities along X-direction for current file:
VXA = file['VXA']

#Extract the first timestep, the initial velocity, and divide it by its magnitude to get the direction (1 or -1)
ivel = (VXA[:, 0]/abs(VXA[0, 0]))
ivel = ((ivel + 1)/2).astype('int')

X, Y = transform_to_relative(all_Xpos, all_Ypos, ivel)

processed = 1

for i in range(1, file_num):
    
    file = scipy.io.loadmat(folder_name + folder[0])

    #Initialize the array for the XA and YA positions
    all_Xpos = np.array(file['XA']).T
    all_Ypos = np.array(file['YA']).T

    #Get the Velocities along X-direction for current file:
    VXA = file['VXA']

    #Extract the first timestep, the initial velocity, and divide it by its magnitude to get the direction (1 or -1)
    ivel = (VXA[:, 0]/abs(VXA[0, 0]))
    ivel = ((ivel + 1)/2).astype('int')

    X_curr, Y_curr = transform_to_relative(all_Xpos, all_Ypos, ivel)
    
    X = np.concatenate((X, X_curr), axis = 0)
    Y = np.concatenate((Y, Y_curr), axis = 0)
    
    processed += 1
    print("Files read = ", processed)
    clear_output(wait=True)

X.shape, Y.shape

((420420, 82), (420420, 1))

## MODEL:

In [7]:
layer_nodes = [20, 50, 80, 50, 1]
#layer_nodes = [800, 2000, 3200, 2000, 1]
dropout_probs = [0, 0.1, 0.2, 0.1, 0]

#Activations:
sigmoid = tf.keras.activations.sigmoid
relu = tf.keras.activations.relu
tanh = tf.keras.activations.tanh

activations = [tanh, tanh, tanh, tanh, tanh]
inputs = tf.keras.layers.Input(shape = (82,))
layer_output = tf.keras.layers.Dense(units = layer_nodes[0], activation = 'tanh')(inputs)
layer_output = tf.keras.layers.Dropout(dropout_probs[0])(layer_output)
layer_output = tf.keras.layers.BatchNormalization()(layer_output)

for i in range(1, len(layer_nodes)-1):
    layer_output = tf.keras.layers.Dense(units = layer_nodes[i], activation = activations[i])(layer_output)
    layer_output = tf.keras.layers.Dropout(dropout_probs[i])(layer_output)
    layer_output = tf.keras.layers.BatchNormalization()(layer_output)
    
layer_output = tf.keras.layers.Dense(units = layer_nodes[-1], activation = activations[-1])(layer_output)

In [10]:
relative_model = tf.keras.Model(inputs = inputs, outputs = layer_output)
relative_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 82)]              0         
                                                                 
 dense_5 (Dense)             (None, 20)                1660      
                                                                 
 dropout_4 (Dropout)         (None, 20)                0         
                                                                 
 batch_normalization_4 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 50)                1050      
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                           

In [11]:
relative_model.compile(optimizer = tf.keras.optimizers.Adam(), 
              loss = 'mean_squared_error',
             metrics = [tf.keras.metrics.Precision()])

In [13]:
history = relative_model.fit(x = X, y = Y, batch_size = 50, epochs = 5)

Epoch 1/5
  421/84084 [..............................] - ETA: 15:45 - loss: 2.7762e-06 - precision: 1.0000

KeyboardInterrupt: 

In [14]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [15]:
relative_model.save('relative_model.h5')

# -----------------------------------------------------------------------------------------------------------------

In [11]:
all_Y.shape

(42042, 1)

In [73]:
arr = np.array([])
arr = np.reshape(np.append(arr, np.array([1,2,3])), [-1, 3])

array([[1., 2., 3.]])

In [44]:
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
new_arr = np.delete(arr, 1, 1)
new_arr

array([[ 1,  3,  4],
       [ 5,  7,  8],
       [ 9, 11, 12]])

In [42]:
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
np.reshape(arr.T[2], [arr.shape[0], -1])

array([[ 3],
       [ 7],
       [11]])

In [36]:
arr.T[2]

array([ 3,  7, 11])

In [22]:
np.reshape(arr[:][2], [arr.shape[0], -1])

ValueError: cannot reshape array of size 4 into shape (3,newaxis)